In [1]:
import sys
import numpy as np
from functools import reduce
import sounddevice as sd

from pam import SoundCommunication

In [2]:
import reedsolo as rs

In [3]:
# sampling rate
FS = 44100

In [4]:
def pad(W, padding=0.5):
    return np.hstack((np.zeros(int(FS*padding)), W, np.zeros(int(FS*padding))))

In [5]:
def encode_rs(text):

    num_parity = 80
    assert len(text) < 255 - num_parity
    msg_bytes = text.encode('ascii')
    
    msg_bytes = b'\x00' * (255 - num_parity - len(msg_bytes)) + msg_bytes

    codec = rs.RSCodec(num_parity)
    return codec.encode(msg_bytes)

In [6]:
text = """
codec = rs.RSCodec(num_parity)
msg_bytes = codec.decode(msg_bytes_rs)
while msg_bytes[0] == 0:
msg_bytes = msg_bytes[1:]
return msg_bytes
"""

In [7]:
def decode_rs(msg_bytes_rs):
    num_parity = 80

    codec = rs.RSCodec(num_parity)
    msg_bytes = codec.decode(msg_bytes_rs)
    while msg_bytes[0] == 0:
        msg_bytes = msg_bytes[1:]
    return msg_bytes

In [8]:
comm = SoundCommunication(FS, 200, 1000, 2000, msgSymLen=255*8)
from testing import Tester
t = Tester(comm)


In [9]:
r = encode_rs(text)
S = comm.send(r)
W = t.addBandpassNoise(S, 4, 0)
rw = comm.decode(W)
t.diff(r,rw)

('00000000000000000X00000000000000000X0000000000000000X000000X000X000000XX00000X000000000000000000000000X00000000000000000X0000000000000X00X000XX0X000000000000000X0000X0000000000X0000000XX0000000000000000XX0000000X00X0000000000X0000000000X000000X00000X00000000000X000X00000000X0000X00000000XX00101001100011011011110110010001100101X110001100100000X011110100X0000001110010011100110X10X110010100100101001101XX0X11011011110X100100011X0101011000110010100001101X10011101010110110101X111110111000001100X01011X001001101001X1110100011X10010010100100001X10011011010111XX11011001110101111101100010011X1001XX110X00X1100101011X00X10010000000X1110100X00X0001X0001X0X10111X01100X000110010101100011001011X001XX0100011001010X10001101101111011001000110010100101000XX10110101110X1101100X1101X1111X0110001001111X010111X10001100101011100110X011111011X001001110011001010010X001010011101110110X00001101001011X1100011001010010000X0110110101X100110110011XX10111110110001001111001X1110X0001100101011100X10101101100110000010111

In [24]:
res = decode_rs(rw)